In [1]:
import os
import sys
sys.path.append('..')

import pandas as pd

from chatbot_app.chatbot import ChatBot
from vector_service.vector_db import VectorDatabase
from src.utils.paths import get_project_path

/home/students/krotkov/SciNanoAI/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vec = VectorDatabase(
    db_path=os.path.join(get_project_path(), "db", "intfloat_multilingual-e5-large"),
    model_name="intfloat/multilingual-e5-large"
    )

In [3]:
chatbot = ChatBot(llm_model="openai/gpt-4o-2024-11-20")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [9]:
print(chatbot.generate_response('какие материалы использовали для изготовления структурированного каркаса для роста клетоточных культур?'))

Для изготовления структурированных каркасов для роста клеточных культур использовались различные материалы, которые можно разделить на несколько категорий в зависимости от их состава и свойств. Основные материалы, упомянутые в предоставленном контексте, включают:

### 1. **Гибридные органо-неорганические материалы**
- **Материалы на основе оксидов титана и циркония (Ti- и Zr-based hybrids)**: Эти материалы использовались для создания трёхмерных каркасов с помощью технологии двухфотонной полимеризации (Two-Photon Polymerization, TPP). Оптимальные составы, такие как Ti:MAA = 1:3 и Zr:MAA = 1:3, обеспечивали баланс между органической и неорганической составляющими. Органическая часть материала создаёт мягкую матрицу для роста клеток, а неорганическая обеспечивает механическую стабильность и жёсткость структуры [2_12_Полина_Металл_Psycharakis_2011_Biomed._Mater._6_045008.pdf].

- **Sol-gel гибриды**: Фоточувствительные материалы на основе кремния-циркония и кремния-титана применялись для с

In [5]:
documents = chatbot.get_relevant_documents('Как влияет обработка поверхности на свойства RGD-паттернов и их взаимодействие с клетками?')

context_parts = []
for doc in documents:
    content = doc.get("content", "")
    metadata = doc.get("metadata", {})
    filename = metadata.get("filename", "неизвестный_файл")
    context_parts.append(f"{content} [{filename}]")

context = "\n\n".join(context_parts)


In [7]:
documents

[{'content': 'Impact of RGD micro-patterns on cell adhesion',
  'metadata': {'detection_class_prob': 0.7506611347198486,
   'coordinates': {'points': [[118.50323486328125, 462.8606],
     [118.50323486328125, 500.2206000000001],
     [955.86669921875, 500.2206000000001],
     [955.86669921875, 462.8606]],
    'system': 'PixelSpace',
    'layout_width': 1654,
    'layout_height': 2205},
   'last_modified': '2024-11-22T22:03:22',
   'filetype': 'application/pdf',
   'languages': ['eng'],
   'page_number': 1,
   'file_directory': '/home/ccinfochem/Documents/krotkov/SciNanoAI/data/Статьи для базы данных/Оригинальные статьи /Наноструктуры и клетки/Влияние наноструктурирования на клетки/Клеточные силы',
   'filename': '18_12_Sbytov.pdf',
   'data_source': {'record_locator': {'path': '/home/ccinfochem/Documents/krotkov/SciNanoAI/data/Статьи для базы данных/Оригинальные статьи /Наноструктуры и клетки/Влияние наноструктурирования на клетки/Клеточные силы/18_12_Sbytov.pdf'},
    'date_modified':

In [5]:
# for mod in ["openai/gpt-4o-mini", "openai/gpt-4o-2024-11-20"]:

#     ans = []

#     chatbot = ChatBot(llm_model=mod)

#     for i in range(len(df)):
#         print(i)
#         resp = chatbot.generate_response(df.Запрос.iloc[i])

#         ans.append(resp)

#     df[mod] = ans

# # df.to_csv('10_val.csv', index=False)


In [70]:
data = pd.read_csv("10_val.csv")
data

,Запрос,openai/gpt-4o-mini,openai/gpt-4o-2024-11-20,Модельный ответ
0,какие конкретные геометрии 3D-каркасов были из...,"В предоставленной информации упоминается, что ...",На основании предоставленного контекста можно ...,с использованием материала SZ2080 были напечат...
1,какие материалы использовали для изготовления ...,Для изготовления структурированного каркаса дл...,"На основании предоставленного контекста, для и...",для изготовления каркасов использовали два раз...
2,какую архитектуру глубокого обучения можно исп...,Для моделирования и оптимизации геометрических...,Для моделирования и оптимизации геометрических...,модель многозадачного глубокого обучения. Моде...
3,"как различная глубина наноям (80, 220 и 333 нм...","Различная глубина наноям (80, 220 и 333 нм) ок...","Различная глубина наноям (80, 220 и 333 нм) ок...","различная глубина наноям (80, 220 и 333 нм) ок..."
4,как изменения в механических условиях могут по...,Изменения в механических условиях окружающей с...,Изменения в механических условиях внешней сред...,"клетки используют механосенсорные пути, активи..."
5,"какие факторы, помимо топографии, могут влиять...","Помимо топографии, на поведение остеопогенитор...","Помимо топографии, на поведение остеопогенитор...",на поведение остеопогениторных клеток помимо т...
6,какие механизмы клеточной адгезии активируются...,При взаимодействии клеток с наноструктурирован...,При взаимодействии клеток с наноструктурирован...,при взаимодействии с наноструктурированными по...
7,Какое влияние нанопаттерны и микропаттерны мог...,Нанопаттерны и микропаттерны оказывают значите...,Нанопаттерны и микропаттерны оказывают значите...,Нанопаттерны могут увеличивать площадь поверхн...
8,Какие ограничения могут возникнуть в процессе ...,Автоматизация обработки текстов и изображений ...,Автоматизация обработки текстов и изображений ...,В процессе автоматизации обработки текстов и и...
9,Как динамическое микропаттернирование на нанос...,Динамическое микропаттернирование на нанострук...,Динамическое микропаттернирование на нанострук...,"Клетки, находясь на подобных наноструктурах, п..."


In [74]:
references = data["Модельный ответ"]
predictions = data["openai/gpt-4o-mini"]

In [75]:
import evaluate

# ============== ROUGE ==============
rouge_metric = evaluate.load("rouge")
rouge_result = rouge_metric.compute(predictions=predictions, references=references)
print("ROUGE Result:", rouge_result)

# ============== BERTScore ==============
bertscore_metric = evaluate.load("bertscore")
bertscore_result = bertscore_metric.compute(
    predictions=predictions,
    references=references,
    model_type="bert-base-multilingual-cased"
)
print("BERTScore Result:", bertscore_result)


ROUGE Result: {'rouge1': 0.08282828282828283, 'rouge2': 0.04475806451612903, 'rougeL': 0.08282828282828283, 'rougeLsum': 0.08282828282828283}


BERTScore Result: {'precision': [0.650126576423645, 0.623824954032898, 0.5910773277282715, 0.7311689853668213, 0.6762884855270386, 0.7313793301582336, 0.7141122817993164, 0.6784335970878601, 0.6327232122421265, 0.6326863765716553], 'recall': [0.6714960336685181, 0.6984882354736328, 0.7024264931678772, 0.8012953996658325, 0.7251221537590027, 0.8026117086410522, 0.7693334817886353, 0.7690648436546326, 0.6895492672920227, 0.7442237734794617], 'f1': [0.6606385111808777, 0.6590486764907837, 0.6419593095779419, 0.7646276950836182, 0.6998544931411743, 0.765341579914093, 0.740695059299469, 0.7209119200706482, 0.6599151492118835, 0.6839374899864197], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.46.3)'}


In [77]:
bleurt_metric = evaluate.load("bleurt", "bleurt-20")
bleurt_result = bleurt_metric.compute(
    predictions=predictions,
    references=references
)

2024-12-16 19:03:42.786619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734365022.801667 2681758 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734365022.806316 2681758 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 19:03:42.823657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1734365114.860334 2681758 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13

InternalError: Graph execution error:

Detected at node bert/embeddings/LayerNorm/moments/SquaredDifference defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/students/krotkov/SciNanoAI/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/students/krotkov/SciNanoAI/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/students/krotkov/SciNanoAI/venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/students/krotkov/SciNanoAI/venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, i

In [ ]:
bleurt_scores = bleurt_result["scores"]
mean_bleurt_score = sum(bleurt_scores) / len(bleurt_scores)